In [1]:
import torch
import numpy as np

In [ ]:
# number of samples - 5000
# number of shapes - 101 (0-100)
# number of operations - 6 (+, -, *, /, &, =)

# vocab size - 107


In [6]:
# rule 0

file = open("isolated_dataset.txt", "w")

for i in range(50, 100):
    file.write(f'{i} + {100 - i} = 100\n\n')
    # print(f'{i} + {100 - i} = 100')

for i in range(1, 100):
    if i < 50:
        file.write(f'{i} - {50 + i} = -50\n\n')
        # print(f'{i} - {50 + i} = -50')
    elif i < 99:
        file.write(f'{i} - {i - 50} = 50\n\n')
        # print(f'{i} - {i - 50} = 50')
    elif i == 99:
        file.write(f'{i} - {i - 50} = 50')
    
file.close()

In [2]:
"""Byte pair encoding utilities"""
import json
import regex as re
from functools import lru_cache

@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


class Encoder:
    def __init__(self, encoder, bpe_merges, errors='replace'):
        self.encoder = encoder
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.errors = errors  # how to handle errors in decoding
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}

        # Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors=self.errors)
        return text


def get_encoder():
    with open('/root/code/team/position_importance_test/pit/gpt-2-Pytorch-train/GPT2/encoder.json', 'r') as f:
        encoder = json.load(f)
    with open('/root/code/team/position_importance_test/pit/gpt-2-Pytorch-train/GPT2/vocab.bpe', 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    return Encoder(encoder=encoder, bpe_merges=bpe_merges)


In [9]:
enc = get_encoder()
print(enc.encode("109"))
# print(enc.encode("3 "))
# print(enc.encode(" 3"))
# print(enc.encode("-"))
# print(enc.encode(" -"))
# print(enc.encode(" "))
# print(enc.encode(" "))
# print(enc.encode(" "))
# print(enc.encode("    "))
# print(enc.encode("    "))


[14454]


In [8]:
import os, glob
import tqdm

def load_dataset(enc, path, combine):
    paths = []

    # Simple file
    if os.path.isfile(path):
        paths.append(path)

    # Directory
    elif os.path.isdir(path):
        for (dirpath, _, fnames) in os.walk(path):
            for fname in fnames:
                paths.append(os.path.join(dirpath, fname))

    # Assume glob
    else:
        paths = glob.glob(path)

    # filter paths
    paths = [p for p in paths if '.DS_Store' not in p]

    token_chunks = []
    raw_text = ''
    for path in tqdm.tqdm(paths):

        if path.endswith('.npz'):

            # Pre-encoded
            with np.load(path) as npz:
                for item in npz.files:
                    token_chunks.append(npz[item])
        else:

            # Plain text
            with open(path, mode='r', encoding='utf-8') as fp:
                raw_text += fp.read()

            if len(raw_text) >= combine:
                tokens = np.stack(enc.encode(raw_text))
                token_chunks.append(tokens)
                raw_text = ''
            else:
                raw_text += '<|endoftext|>'

    if raw_text:
        tokens = np.stack(enc.encode(raw_text))
        token_chunks.append(tokens)

    return token_chunks

In [35]:
# enc = get_encoder()
chunks = load_dataset(encoder, './isolated_dataset.txt', 1000)

  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'encode'

In [31]:
chunks[0].size

1090

In [32]:
for i in range(chunks[0].size):    
    if chunks[0][i] == 50256:
        print(i)

In [36]:
# Encoder

encoder = {"0": 0}
operation = ["+", "-", "*", "/", "&", "="]

num = 1
for i in range(100):
    encoder[f"{i}"] = num
    num += 1

for i in operation:
    encoder[i] = num
    num += 1

encoder["<|endoftext|>"] = num
num += 1

# encoder


In [37]:
with open("simple_encoder.json", "w") as file:
    json.dump(encoder, file)